In [1]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/sergi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/sergi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import os, re
from nltk.tokenize import word_tokenize

import nltk

from rake_nltk import Rake

import yake

import spacy, pytextrank

In [3]:
class Extractor:
    def __init__(self, language, max_tokens):
        self.language = language
        self.max_tokens = max_tokens

    def extract_terms(self, text):
        raise NotImplementedError("extract_terms method must be implemented in subclass")

    def extract_terms_with_span(self, text):
        terms = self.extract_terms(text)
        terms_with_span = self.find_term_span(text, terms)
        return terms_with_span

    #if some of the kwargs do not exist, they will simply be ignored
    def extract_terms_without_overlaps(self, text, kwargs):
        self.kwargs = kwargs
        terms_with_span = self.extract_terms_with_span(text)
        terms_without_overlaps = self.rmv_overlaps(terms_with_span)
        return terms_without_overlaps

    def postprocess_terms(self, terms):
        new_terms = self.rmv_meaningless(terms)
        new_terms = self.rmv_stopwords(new_terms)
        new_terms = self.rmv_stopwords(new_terms)
        new_terms = self.rmv_nonalnum_characters(new_terms)
        new_terms = self.rmv_nonalnum_characters(new_terms)
        new_terms = self.rmv_meaningless(new_terms)
        return new_terms

    def rmv_meaningless(self, terms):
        self.stopwords = nltk.corpus.stopwords.words(self.language)
        new_terms = [t for t in terms if (t[0] not in self.stopwords and len(t[0]) >= 2 and not t[0].isdigit())]
        return new_terms

    def rmv_stopwords(self, terms):
        new_terms = terms
        self.stopwords = nltk.corpus.stopwords.words(self.language)
        for word in self.stopwords:
            new_terms = [(t[0][(len(word)+1):], t[1] + len(word) + 1, t[2], t[3], t[4]) if (t[0].lower().startswith(word.lower() + " ")) else t for t in new_terms]
            new_terms = [(t[0][:-(len(word)+1)], t[1], t[2] - len(word) - 1, t[3], t[4]) if (t[0].lower().endswith(" " + word.lower())) else t for t in new_terms]
        return new_terms

    def rmv_nonalnum_characters(self, terms):
        new_terms = [(t[0][1:], t[1] + 1, t[2], t[3], t[4]) if not t[0][0].isalnum() else t for t in terms]
        new_terms = [(t[0][:-1], t[1], t[2] - 1, t[3], t[4]) if not t[0][-1].isalnum() else t for t in new_terms]
        return new_terms

    @staticmethod
    def find_term_span(text, terms):
        spans = []
        for t,score in terms:
          term = re.escape(t)
          patron = r'\b' + term + r'\b'
          coincidencias = re.finditer(patron, text, re.IGNORECASE)
          span = [(t, coincidencia.start(), coincidencia.end()-1, score) for coincidencia in coincidencias]
          spans.extend(span)
        return spans

    @staticmethod
    def rmv_overlaps(keywords):
      ent = [kw[0] for kw in keywords]
      pos = [kw[1:3] for kw in keywords]
      score = [kw[3:] for kw in keywords]
      updated_keywords = []
      repeated_words = []
      for i in range(len(ent)):
        overlap = False
        if pos[i] in repeated_words:
          overlap = True
        else:
          repeated_words.append(pos[i])
          for k in range(len(ent)):
            if (((pos[i][0] >= pos[k][0]) and (pos[i][1] < pos[k][1])) or ((pos[i][0] > pos[k][0]) and (pos[i][1] <= pos[k][1]))):
              overlap = True
        if (not overlap):
          kw = (ent[i],pos[i][0],pos[i][1]) + score[i]
          updated_keywords.append(kw)
      return updated_keywords

In [4]:
class RakeExtractor(Extractor):
    def __init__(self, language, max_tokens):
        super().__init__(language, max_tokens)
        self.stopwords = nltk.corpus.stopwords.words(language)
        self.extractor = Rake(stopwords=self.stopwords, language=language)

    def extract_terms(self, text):
        filtered_kwargs = {key: value for key, value in self.kwargs.items() if key in self.extractor.extract_keywords_from_text.__code__.co_varnames}
        self.extractor.extract_keywords_from_text(text, **filtered_kwargs)
        terms = self.extractor.get_ranked_phrases_with_scores()
        terms = [(kw,score) for score,kw in terms if (len(word_tokenize(kw, language=self.language)) <= self.max_tokens)]
        return terms

In [5]:
class YakeExtractor(Extractor):
    def __init__(self, language, max_tokens):
        super().__init__(language, max_tokens)
        if language=='spanish':
            self.extractor = yake.KeywordExtractor() #aqui habria que poner top=70 por ejemplo
        else:
            raise ValueError("Expected spanish language. Other languages not recognised")

    def extract_terms(self, text):
        filtered_kwargs = {key: value for key, value in self.kwargs.items() if key in self.extractor.extract_keywords.__code__.co_varnames}
        keywords = self.extractor.extract_keywords(text, **filtered_kwargs)
        terms = [(kw,score) for kw, score in keywords if (len(word_tokenize(kw)) <= self.max_tokens)]
        return terms

In [6]:
class TextRankExtractor(Extractor):
    def __init__(self, language, max_tokens):
        super().__init__(language, max_tokens)
        if language=='spanish':
            self.extractor = spacy.load("es_core_news_sm")
            self.extractor.add_pipe("textrank")
        else:
            raise ValueError("Expected spanish language. Other languages not recognised")

    def extract_terms(self, text):
        filtered_kwargs = {key: value for key, value in self.kwargs.items() if key in self.extractor.__call__.__code__.co_varnames}
        doc = self.extractor(text, **filtered_kwargs)
        terms = []
        for phrase in doc._.phrases:
          if (len(word_tokenize(phrase.text)) <= self.max_tokens):
            terms.append((phrase.text, phrase.rank))
        return terms

In [7]:
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer

/mnt/c/Users/Sergi/Documents/BioTermCategorizer/.env_biotermcategorizer/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
class KeyBertExtractor(Extractor):
    def __init__(self, language, max_tokens):
        super().__init__(language, max_tokens)
        sentence_model = self.initialize_sentence_model()
        self.extractor = KeyBERT(model=sentence_model)

    def initialize_sentence_model(self):
        path = '/mnt/c/Users/Sergi/Desktop/BSC/spanish_sapbert_models/sapbert_15_parents_1epoch'
        sentence_model = SentenceTransformer(path)
        return sentence_model

    def extract_terms(self, text):
        filtered_kwargs = {key: value for key, value in self.kwargs.items() if key in self.extractor.extract_keywords.__code__.co_varnames}
        specified_params = ['keyphrase_ngram_range', 'stop_words', 'use_mmr']
        for param in specified_params:
            if param in filtered_kwargs:
                del filtered_kwargs[param]
                print("Warning. The parameter " + param + " can not be changed from its given value.")
        keywords_all = list()
        for i in range(self.max_tokens):
            keywords_i = self.extractor.extract_keywords(text, keyphrase_ngram_range=(1, i+1), stop_words=None, use_mmr=True, **filtered_kwargs)
            keywords_all.extend(keywords_i)
        terms = sorted(keywords_all, key=lambda x: x[1], reverse=True)
        return terms

In [9]:
class TermExtractor:
    def __init__(self,
                 extraction_methods=["textrank"], 
                 categorizer_method="setfit", 
                 language="spanish", 
                 max_tokens=3, 
                 join=False, 
                 postprocess=True,
                 n=1, 
                 thr_setfit=0.5,
                 thr_transformers=-1,
                 n_clusters=None,
                 categorizer_model_path=None,
                 output_path="./trained_model", 
                 clustering_model="cambridgeltl/SapBERT-from-PubMedBERT-fulltext",
                 classifier_model="/mnt/c/Users/Sergi/Desktop/BSC/spanish_sapbert_models/sapbert_15_noparents_1epoch",
                 **kwargs,
                ):
        self.extraction_methods = extraction_methods
        self.extractors = self.initialize_keyword_extractors(language, max_tokens)
        self.categorizer_method = categorizer_method
        self.categorizer = self.initialize_categorizers(n, thr_setfit, thr_transformers, n_clusters, categorizer_model_path, output_path, clustering_model, classifier_model)
        self.join = join
        self.postprocess = postprocess
        self.kwargs = kwargs
    
    def __call__(self, text):
        self.extract_terms(text, self.join, self.postprocess)
        self.categorize_terms()
        
    def initialize_keyword_extractors(self, language, max_tokens):
        keyword_extractors = {}
        
        if 'rake' in self.extraction_methods:
            keyword_extractors["rake"] = RakeExtractor(language, max_tokens)
        
        if 'yake' in self.extraction_methods:
            keyword_extractors["yake"] = YakeExtractor(language, max_tokens)
        
        if 'textrank' in self.extraction_methods:
            keyword_extractors["textrank"] = TextRankExtractor(language, max_tokens)

        if 'keybert' in self.extraction_methods:
            keyword_extractors["keybert"] = KeyBertExtractor(language, max_tokens)

        if not keyword_extractors:
            raise ValueError("No extraction method called {}".format(self.extraction_methods))
        
        return keyword_extractors

    def extract_terms(self, text, join, postprocess):
        try:
            all_terms = []
            for key, extractor in self.extractors.items():
                terms = extractor.extract_terms_without_overlaps(text, kwargs=self.kwargs)
                terms = [term + (key,) for term in terms]
                all_terms.extend(terms)
            if join:
                all_terms = list(self.extractors.values())[0].rmv_overlaps(all_terms)
            if postprocess:
                all_terms = list(self.extractors.values())[0].postprocess_terms(all_terms)
                all_terms = list(self.extractors.values())[0].rmv_overlaps(all_terms)
            self.keywords = [Keyword(text=i[0], extraction_method=i[4], ini=i[1], fin=i[2], score=i[3]) for i in all_terms]
        except:
            raise AttributeError("A list of extractors must be provided")

    def initialize_categorizers(self, n, thr_setfit, thr_transformers, n_clusters, model_path, output_path, clustering_model, classifier_model):
        if 'transformers' == self.categorizer_method:
            categorizer = TransformersClassifier(n, thr_transformers, model_path, output_path, classifier_model)
            
        elif 'setfit' == self.categorizer_method:
            categorizer = SetFitClassifier(n, thr_setfit, model_path, output_path, classifier_model)

        elif 'clustering' == self.categorizer_method:
            if n_clusters is None:
                raise TypeError("TermExtractor.__init__() missing 1 required positional argument: 'n_clusters' when selecting the Clustering algorithm")
            categorizer = Clustering(n_clusters, model_path, output_path, clustering_model)
        else:
            raise ValueError("No categorizer method called {}".format(self.categorizer_method))
        return categorizer

    def categorize_terms(self):
        try:
            if self.categorizer_method == 'clustering':
                list_of_keywords = [kw.text for kw in self.keywords]
                clusters = self.categorizer.predict_clusters(list_of_keywords)
                for kw in self.keywords:
                    kw.label = clusters[kw.text]
                    kw.categorization_method = self.categorizer_method
            else:
                for kw in self.keywords:
                    kw.label = self.categorizer.compute_predictions(kw.text)
                    kw.categorization_method = self.categorizer_method
        except:
            raise AttributeError("A categorizer method must be provided")

    def train_classifier(self, trainX, trainY, testX, testY, mcm=False, classification_report=False, **kwargs):
        self.categorizer.initialize_model_body(trainY)
        metrics = self.categorizer.train_evaluate(trainX, trainY, testX, testY, mcm, classification_report, **kwargs)
        print(metrics)

    def train_clustering(self, trainX, **kwargs):
        self.categorizer.train_clusters(trainX, **kwargs)

In [10]:
class Keyword:
    def __init__(self, text, extraction_method, ini, fin, score):
        self.text = text
        self.extraction_method = extraction_method
        self.score = score
        self.span = [ini, fin]
        self.categorization_method = None
        self.label = None

    def __repr__(self):
        return f"<Keyword(text='{self.text}', span='{self.span}', extraction method='{self.extraction_method}', score='{self.score}', categorization method='{self.categorization_method}', class='{self.label}')>"

In [11]:
from sklearn.preprocessing import MultiLabelBinarizer

In [12]:
import evaluate
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_fscore_support, confusion_matrix, classification_report
from collections import Counter

In [13]:
class Categorizer:
    def __init__(self, n, threshold, model_path, output_path, classifier_model):
        self.n = n
        self.threshold = threshold
        self.labels = ['ACTIVIDAD', 'COMUNIDAD', 'DEPARTAMENTO', 'ENFERMEDAD', 'FAC_GEN','FAC_NOM', 'FARMACO', 'GEO_GEN', 'GEO_NOM', 'GPE_GEN', 'GPE_NOM','HUMAN', 'IDIOMA', 'MORFOLOGIA_NEOPLASIA', 'NO_CATEGORY','PROCEDIMIENTO', 'PROFESION', 'SINTOMA', 'SITUACION_LABORAL','SPECIES', 'TRANSPORTE']
        self.output_path = output_path
        self.classifier_model = classifier_model
        self.model = self.initialize_pretrained_model(model_path)

    def evaluate_model(self, y_pred, y_test, mcm, classification_report):
        if classification_report:
            self.classification_report(y_pred, y_test)
        if mcm:
            self.mcm_heatmap(y_pred, y_test)
        return self.compute_metrics(y_pred, y_test)

    def lambda_evaluate_model(self, mcm, classification_report):
        return lambda y_pred, y_test: self.evaluate_model(y_pred, y_test, mcm, classification_report)

    def compute_metrics(self, y_pred, y_test):
        multilabel_f1_metric = evaluate.load("f1", "multilabel")
        multilabel_accuracy_metric = evaluate.load("accuracy", "multilabel")
        f1 = multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="micro")["f1"]
        accuracy = multilabel_accuracy_metric.compute(predictions=y_pred, references=y_test)["accuracy"]

        y_pred = np.array(y_pred)
        y_test = np.array(y_test)
        
        no_label_samples = []
        for idx, pred in enumerate(y_pred):
            if np.all(pred == 0):
                true_labels = [self.labels[i] for i, value in enumerate(y_test[idx]) if value == 1]
                no_label_samples.extend(true_labels)
    
        label_counts = Counter(no_label_samples)
        label_counts_dict = dict(label_counts)
        return {"f1": f1, "accuracy": accuracy, "Classes with no given label": label_counts_dict}

    def classification_report(self, y_pred, y_test):
        y_pred = np.array(y_pred)
        y_test = np.array(y_test)

        print(classification_report(y_test, y_pred, target_names=self.labels))

    def mcm_heatmap(self, y_pred, y_test):
        y_pred = np.array(y_pred)
        y_test = np.array(y_test)
        
        samples_with_predictions = np.any(y_pred, axis=1)
    
        # Filter y_pred and y_test to include only samples with predictions
        y_pred_with_predictions = y_pred[samples_with_predictions]
        y_test_with_predictions = y_test[samples_with_predictions]
    
        # Initialize the confusion matrix
        confusion_matrix_multi = confusion_matrix(y_test_with_predictions.argmax(axis=1), y_pred_with_predictions.argmax(axis=1), labels=np.arange(len(self.labels)))
    
        # Create a heatmap-style visualization
        plt.figure(figsize=(10, 7))
        plt.imshow(confusion_matrix_multi, interpolation="nearest", cmap=plt.cm.Blues)
        plt.title("Multi-Label Confusion Matrix")
        plt.colorbar()
    
        # Label the axes
        tick_marks = np.arange(len(self.labels))
        plt.xticks(tick_marks, self.labels, rotation=45)
        plt.yticks(tick_marks, self.labels)
    
        # Display the values inside the cells
        for i in range(len(self.labels)):
            for j in range(len(self.labels)):
                plt.text(j, i, str(confusion_matrix_multi[i, j]), ha="center", va="center", color="white" if confusion_matrix_multi[i, j] > confusion_matrix_multi.max() / 2 else "black")
    
        plt.tight_layout()
        plt.ylabel("True Labels")
        plt.xlabel("Predicted Labels")
        plt.show()

In [14]:
from setfit import SetFitModel, SetFitTrainer
import torch

In [15]:
import pandas as pd
from datasets import Dataset

In [16]:
class SetFitClassifier(Categorizer):
    def __init__(self, n, threshold, model_path, output_path, classifier_model):
        super().__init__(n, threshold, model_path, output_path, classifier_model)
    
    def initialize_pretrained_model(self, model_path):
        if model_path is None:
            path = '/mnt/c/Users/Sergi/Desktop/BSC/modelos_entrenados/SetFit/noparents_sp'
            model = SetFitModel.from_pretrained(path)
        else:
            model = SetFitModel.from_pretrained(model_path)
        return model

    def compute_predictions(self, mention):
        embeddings = self.model.model_body.encode([mention], normalize_embeddings=self.model.normalize_embeddings, convert_to_tensor=True)
        predicts = self.model.model_head.predict_proba(embeddings)
        predscores = {self.labels[i]: arr[:,1].tolist()[0] for i, arr in enumerate(predicts)}
        top_n_labels = sorted(predscores, key=predscores.get, reverse=True)[:self.n]
        filtered_labels = [label for label in top_n_labels if predscores[label] > self.threshold]
        return filtered_labels

    def initialize_model_body(self, trainY):
        self.model = SetFitModel.from_pretrained(self.classifier_model, multi_target_strategy="multi-output")

    def train_evaluate(self, trainX, trainY, testX, testY, mcm, classification_report, **kwargs):
        train_dataset, test_dataset = self.prepare_data(trainX, trainY, testX, testY)
        evaluate_with_params = self.lambda_evaluate_model(mcm, classification_report)
        specified_params = ['metric', 'num_iterations']
        for param in specified_params:
            if param in kwargs:
                del kwargs[param]
                print("Warning. The parameter " + param + " can not be changed from its given value.")
        trainer = SetFitTrainer(model=self.model, train_dataset=train_dataset, eval_dataset=test_dataset, metric=evaluate_with_params, num_iterations=5, **kwargs)
        trainer.train()
        metrics = trainer.evaluate()
        self.model.save_pretrained(self.output_path)
        return metrics

    def prepare_data(self, trainX, trainY, testX, testY):
        trainY = [{i} for i in trainY]
        testY = [{i} for i in testY]
        mlb = MultiLabelBinarizer()
        mlb.fit_transform(trainY)
        self.labels = [i for i in mlb.classes_]
        train_dataset = Dataset.from_dict({"text": trainX, "label": mlb.fit_transform(trainY)})
        test_dataset = Dataset.from_dict({"text": testX, "label": mlb.transform(testY)})
        return train_dataset, test_dataset

In [17]:
import pandas as pd
import torch
from torch.utils.data import TensorDataset
from sklearn.preprocessing import MultiLabelBinarizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

In [18]:
class TransformersClassifier(Categorizer):
    def __init__(self, n, threshold, model_path, output_path, classifier_model):
        super().__init__(n, threshold, model_path, output_path, classifier_model)
        
    def initialize_pretrained_model(self, model_path):
        self.mlb = MultiLabelBinarizer()
        self.mlb.fit([self.labels])
        self.tokenizer = AutoTokenizer.from_pretrained(self.classifier_model)
        if model_path is None:
            path = '/mnt/c/Users/Sergi/Desktop/BSC/modelos_entrenados/Transformers/noparents_sp'        
            model = AutoModelForSequenceClassification.from_pretrained(path)
        else:
            model = AutoModelForSequenceClassification.from_pretrained(model_path)
        return model

    def compute_predictions(self, mention):
        tokenized_mention = self.tokenizer(mention, return_tensors='pt', padding=True, truncation=True)
        with torch.no_grad():
            output = self.model(**tokenized_mention)
        logits = output.logits
        predscores = {label: score for label, score in zip(self.labels, logits.tolist()[0])}
        top_n_labels = sorted(predscores, key=predscores.get, reverse=True)[:self.n]
        filtered_labels = [label for label in top_n_labels if predscores[label] > self.threshold]
        return filtered_labels
    
    def initialize_model_body(self, trainY):
        self.find_labels(trainY)
        self.tokenizer = AutoTokenizer.from_pretrained(self.classifier_model)
        self.model = AutoModelForSequenceClassification.from_pretrained(self.classifier_model, num_labels=self.num_labels, problem_type="multi_label_classification")

    #Im doing it as if I received a dataframe with the samples in columns called "text" and "label"
    def train_evaluate(self, trainX, trainY, testX, testY, mcm, classification_report, **kwargs):
        train_dataset = self.prepare_data(trainX, trainY)
        self.train(train_dataset, **kwargs)
        test_dataset = self.prepare_data(testX, testY)
        self.evaluate(test_dataset, mcm, classification_report)
        self.model.save_pretrained(self.output_path)

    def find_labels(self, trainY):
        self.mlb = MultiLabelBinarizer()
        Y = [[i] for i in trainY]
        self.mlb.fit(Y)
        self.labels = [i for i in self.mlb.classes_]
        self.num_labels = len(self.labels)
    
    def prepare_data(self, X, Y):
        tokenized_data = self.tokenizer(X, truncation=True, padding=True, return_tensors="pt", max_length=512)
        label_strings = [[i] for i in Y]
        labels = self.mlb.transform(label_strings)
        labels = torch.tensor(labels, dtype=torch.float32)
        tensordataset = TensorDataset(tokenized_data.input_ids, tokenized_data.attention_mask, labels)
        return tensordataset

    def train(self, train_dataset, **kwargs):
        training_args = TrainingArguments(
            output_dir="./output",  # Output directory
            num_train_epochs=3,     # Number of training epochs
            per_device_train_batch_size=32,  # Batch size per device
            evaluation_strategy="steps",  # Evaluate every steps
            save_steps=500,  # Save checkpoint every 500 steps
            save_total_limit=2,  # Only keep the last 2 checkpoints
            load_best_model_at_end=True,  # Load the best model at the end of training
        )
        specified_params = ['data_collator', 'args','model']
        for param in specified_params:
            if param in kwargs:
                del kwargs[param]
                print("Warning. The parameter " + param + " can not be changed from its given value.")
        self.trainer = Trainer(
            model=self.model,
            args=training_args,
            data_collator=self.collate_fn,  # You can customize data collation if needed
            train_dataset=train_dataset,
            **kwargs,
        )
        self.trainer.train()

    def evaluate(self, testset, mcm, classification_report):
        results = self.trainer.predict(testset)
        max_indices = np.argmax(results.predictions, axis=1)
        preds = np.zeros_like(results.predictions)
        preds[np.arange(len(max_indices)), max_indices] = 1
        metrics = self.evaluate_model(preds, results.label_ids, mcm, classification_report)
        print(metrics)
        
    def collate_fn(self, batch):
        return {
        'input_ids': torch.stack([item[0] for item in batch]),
        'attention_mask': torch.stack([item[1] for item in batch]),
        'labels': torch.stack([item[2] for item in batch])
        }

In [19]:
from sklearn.cluster import KMeans
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch
import pickle

In [20]:
class Clustering(Categorizer):
    def __init__(self, n_clusters, model_path, output_path, clustering_model):
        self.tokenizer = AutoTokenizer.from_pretrained(clustering_model)
        self.model = AutoModel.from_pretrained(clustering_model)
        self.trained = False
        self.n_clusters = n_clusters
        self.output_path = output_path
        self.model_path = model_path
        
    def generate_embeddings(self, mentions):
        tokenized_inputs = self.tokenizer(mentions, padding=True, truncation=True, return_tensors="pt")
        with torch.no_grad():
            outputs = self.model(**tokenized_inputs)
        embeddings = outputs.last_hidden_state
        return embeddings[:, 0, :]

    def train_clusters(self, mentions_train, **kwargs):
        mentions_embeddings_train = self.generate_embeddings(mentions_train)
        specified_params = ['random_state', 'n_init']
        for param in specified_params:
            if param in kwargs:
                del kwargs[param]
                print("Warning. The parameter " + param + " can not be changed from its given value.")
        self.kmeans = KMeans(n_clusters=self.n_clusters, random_state=0, n_init="auto", **kwargs).fit(mentions_embeddings_train)
        self.trained = True
        self.save_trained_model()

        clusters_examples = dict()
        for i in range(max(self.kmeans.labels_)+1):
            instances = []
            for j in range(len(self.kmeans.labels_)):
                if self.kmeans.labels_[j] == i and len(instances) < 5:
                    instances.append(mentions_train[j])
            clusters_examples["Class " + str(i)] = instances
        print(clusters_examples)

    def predict_clusters(self, mentions):
        word_with_cluster = {}
        mentions_embeddings = self.generate_embeddings(mentions)
        if self.model_path is not None and not self.trained:
            self.import_pretrained_model(self.model_path)
            self.trained = True
        elif not self.trained:
            self.train_clusters(mentions)
            self.trained = True
            self.save_trained_model()
        predicted_clusters = self.kmeans.predict(mentions_embeddings)
        for i in range(len(mentions)):
            word_with_cluster[mentions[i]] = predicted_clusters[i]
        return word_with_cluster
    
    def save_trained_model(self):
        with open(self.output_path, 'wb') as file:
            pickle.dump(self.kmeans, file)      

    def import_pretrained_model(self, model_path):
        with open(model_path, 'rb') as file:
            self.kmeans = pickle.load(file)
        self.trained = True

In [21]:
import pandas as pd

file_path = '/mnt/c/Users/Sergi/Desktop/BSC/traindata_classification.tsv'
traindata = pd.read_csv(file_path, sep='\t')
traindata = traindata.sample(frac=1, random_state=42)
traindata_head = traindata.head(100)
traindata_head = traindata_head[traindata_head["label"].isin(["SINTOMA","ENFERMEDAD","SINTOMA","PROCEDIMIENTO"])]

file_path = '/mnt/c/Users/Sergi/Desktop/BSC/testdata_classification.tsv'
testdata = pd.read_csv(file_path, sep='\t')
testdata = testdata.sample(frac=1, random_state=42)
testdata_head = testdata.head(30)
testdata_head = testdata_head[testdata_head["label"].isin(["SINTOMA","ENFERMEDAD","SINTOMA","PROCEDIMIENTO"])]

trainX = traindata_head["text"].values.tolist()
trainY = traindata_head["label"].values.tolist()
testX = testdata_head["text"].values.tolist()
testY = testdata_head["label"].values.tolist()

In [22]:
extractor = TermExtractor(extraction_methods=["textrank"],categorizer_method="setfit")

/mnt/c/Users/Sergi/Documents/BioTermCategorizer/.env_biotermcategorizer/lib/python3.10/site-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.2.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/mnt/c/Users/Sergi/Documents/BioTermCategorizer/.env_biotermcategorizer/lib/python3.10/site-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator MultiOutputClassifier from version 1.2.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [23]:
extractor("La importancia vital de la esterilización de las manos antes de una laparoscopia es esencial si hay gripe")

In [24]:
extractor.keywords

[<Keyword(text='gripe', span='[100, 104]', extraction method='textrank', score='0.2795984421205809', categorization method='setfit', class='['SPECIES']')>,
 <Keyword(text='importancia vital', span='[3, 19]', extraction method='textrank', score='0.09206970100819285', categorization method='setfit', class='['NO_CATEGORY']')>,
 <Keyword(text='laparoscopia', span='[68, 79]', extraction method='textrank', score='0.08772274252666795', categorization method='setfit', class='['PROCEDIMIENTO']')>,
 <Keyword(text='manos', span='[49, 53]', extraction method='textrank', score='0.08182759864003777', categorization method='setfit', class='[]')>,
 <Keyword(text='esterilización', span='[27, 40]', extraction method='textrank', score='0.07223214099675339', categorization method='setfit', class='['PROCEDIMIENTO']')>]

In [23]:
extractor.train_classifier(trainX,trainY,testX,testY)

No sentence-transformers model found with name /mnt/c/Users/Sergi/Desktop/BSC/spanish_sapbert_models/sapbert_15_noparents_1epoch. Creating a new one with MEAN pooling.
model_head.pkl not found in /mnt/c/Users/Sergi/Desktop/BSC/spanish_sapbert_models/sapbert_15_noparents_1epoch, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
Generating Training Pairs: 100%|█████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 200.46it/s]
***** Running training *****
  Num examples = 420
  Num epochs = 1
  Total optimization steps = 27
  Total train batch size = 16
Epoch: 100%|█████████████████████████████████████████████████████████████████████████████| 1/1 [01:45<00:00, 105.66s/it]
***** Running evaluation *****


{'f1': 0.7999999999999999, 'accuracy': 0.6875, 'Classes with no given label': {'ENFERMEDAD': 2, 'SINTOMA': 1}}


In [35]:
traindata_clusters = traindata_head["text"].tolist()
extractor.train_clustering(traindata_clusters, max_iter=200, random_state=42)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Warning. The parameter random_state can not be changed from its given value.
{'Class 0': ['cavernoma', 'lesión', 'tumoración carnosa de consistencia densa que rodeaba la rama del nervio dentario inferior'], 'Class 1': ['retinotoxicidad por vigabatrina', 'Doppler de arterias renales y el ecocardiograma no mostraron alteraciones', 'ferritina y saturación de transferan normales', 'exploración física', 'ganglios linfáticos, siendo los 25 negativos para células tumorales'], 'Class 2': ['metástasis ganglionares retroperitoneales', 'RM craneal', 'MAVD', 'inestabilidad a la marcha', 'antisépticos locales']}


In [ ]:
predclusters = self.kmeans.labels_

clusters_examples = dict()
for i in range(max(predclusters)+1):
    for j in range(len(predclusters)):
        if predclusters[j] == i and len(instances) < 5:
            clusters_examples["Class " + str(i)] = mentions_train[j]

print(clusters_examples)

In [20]:
import sys, os, re
general_path = os.getcwd().split("BioTermCategorizer")[0]+"BioTermCategorizer/"
sys.path.append(general_path+'biotermcategorizer/')


In [36]:
from TermExtractor import TermExtractor

In [37]:
extractor = TermExtractor(extraction_methods=["textrank","rake"])

In [20]:
#sample texts
text1 = "Paciente varón de 35 años con tumoración en polo superior de teste derecho hallada de manera casual durante una autoexploración, motivo por el cual acude a consulta de urología donde se realiza exploración física, apreciando masa de 1cm aproximado de diámetro dependiente de epidídimo, y ecografía testicular, que se informa como lesión nodular sólida en cabeza de epidídimo derecho. Se realiza RMN. Confirmando masa nodular, siendo el tumor adenomatoide de epidídimo la primera posibilidad diagnóstica. Se decide, en los dos casos, resección quirúrgica de tumoración nodular en cola epidídimo derecho, sin realización de orquiectomía posterior. En ambos casos se realizó examen anátomopatológico de la pieza quirúrgica. Hallazgos histológicos macroscópicos: formación nodular de 1,5 cms (caso1) y 1,2 cms (caso 2) de consistencia firme, coloración blanquecina y bien delimitada. Microscópicamente se observa proliferación tumoral constituida por estructuras tubulares en las que la celularidad muestra núcleos redondeados y elongados sin atipia citológica y que ocasionalmente muestra citoplasmas vacuolados, todo ello compatible con tumor adenomatoide de epidídimo."
text2 = "Dos recién nacidos, varón y hembra de una misma madre y fallecidos a los 10 y 45 minutos de vida respectivamente a los que se les realizó examen necrópsico. El primero de los cadáveres, correspondiente a la hembra, fue remitido con el juicio clínico de insuficiencia respiratoria grave con sospecha de Síndrome de Potter con la constatación de oligoamnios severo; nació mediante cesárea urgente por presentación de nalgas y el test de Apgar fue 1/3/7; minutos más tarde falleció. El examen externo permitió observar una tonalidad subcianótica, facies triangular con hendiduras parpebrales mongoloides, micrognatia, raiz nasal ancha y occipucio prominente. El abdomen, globuloso, duro y ligeramente abollonado permitía la palpación de dos grandes masas ocupando ambas fosas renales y hemiabdomenes. A la apertura de cavidades destacaba la presencia de dos grandes masas renales de 10 x 8 x 5,5 cm y 12 x 8 x 6 cm con pesos de 190 y 235 gr respectivamente. Si bien se podía discernir la silueta renal, la superficie, abollonada, presentaba numerosas formaciones quísticas de contenido seroso; al corte dichos quistes mostraban un tamaño heterogéneo siendo mayores los situados a nivel cortical, dando al riñón un aspecto de esponja. Los pulmones derecho e izquierdo pesaban 17 y 15 gr (peso habitual del conjunto de 49 gr) mostrando una tonalidad rojiza uniforme; ambos se encontraban comprimidos como consecuencia de la elevación diafragmática condicionada por el gran tamaño de los riñones. El resto de los órganos no mostraba alteraciones macroscópicas significativas salvo las alteraciones posicionales derivadas de la compresión renal. En el segundo de los cadáveres, el correspondiente al varón, se observaron cambios morfológicos similares si bien el tamaño exhibido por los riñones era aún mayor, con pesos de 300 y 310 gr. El resto de las vísceras abdominales estaban comprimidas contra el diafragma. En ambos casos se realizó un estudio histológico detallado, centrado especialmente en los riñones en los que se demostraron múltiples quistes de distintos tamaños con morfología sacular a nivel cortical. Dichos quistes ocupaban la mayor parte del parénquima corticomedular si bien las zonas conservadas no mostraban alteraciones significativas salvo inmadurez focal. Dichos quistes estaban tapizados por un epitelio simple que variaba desde plano o cúbico. Los quistes medulares, de menor tamaño y más redondeados estaban tapizados por un epitelio de predominio cúbico. Después de las renales, las alteraciones más llamativas se encontraban en el hígado donde se observaron proliferación y dilatación, incluso quística, de los ductos biliares a nivel de los espacios porta. Con tales hallazgos se emitió en ambos casos el diagnóstico de enfermedad poliquística renal autosómica recesiva infantil."
text3 = "Paciente de 64 años, alérgico a penicilina y con recambio valvular aórtico por endocarditis que consultó por aparición de masa peneana de crecimiento progresivo en las últimas semanas. A la exploración física destacaba una formación excrecente y abigarrada en glande, que deformaba meato, con áreas ulceradas cubiertas de fibrina. Se palpaban adenopatías fijas y duras en ambas regiones inguinales. La radiografía de tórax y el TAC abdomino-pélvico confirmaron la presencia de adenopatías pulmonares e inguinales de gran tamaño. Con el diagnóstico de neoplasia de pene, se practicó penectomía parcial con margen de seguridad. La anatomía patológica demostró que se trataba de un sarcoma pleomórfico de pene con diferenciación osteosarcomatosa y márgenes libres de afectación. Se decidió tratamiento con dos líneas de quimioterapia consistente en adriamicina e ifosfamida pero no hubo respuesta. Ingresó de nuevo con recidiva local sangrante de gran tamaño y crecimiento rápido que provocaba obstrucción de meato con insuficiencia renal aguda. Se colocó sonda de cistostomía y se instauró tratamiento con sueroterapia, mejorando la función renal, pero con empeoramiento progresivo del estado general hasta que falleció a los 6 meses del diagnóstico."
text4 = "Mujer de 28 años sin antecedentes de interés que consultó por síndrome miccional con polaquiuria de predominio diurno y cierto grado de urgencia sin escapes urinario. El urocultivo resultó negativo por lo que se indicó tratamiento con anticolinérgicos. Ante la falta de respuesta al tratamiento, se realizó cistografía que fué normal y ecografía renovesical en la que se apreciaban imágenes quísticas parapiélicas, algunas de ellas con tabiques internos y vejiga sin lesiones. Con el fin de precisar la naturaleza de dichos quistes se solicitó TAC-abdominal, que informaba de gran quiste parapiélico en riñón derecho sin repercusión sobre la vía y una masa hipodensa suprarrenal derecha. La resonancia magnética demostró normalidad de la glándula suprarrenal y una lesión quística lobulada conteniendo numerosos septos en su interior que rodeaba al riñón derecho; en la celda renal izquierda existía una lesión de características similares pero de menor tamaño. Los hallazgos eran compatibles con linfangioma renal bilateral. Tras tres años de seguimiento la paciente continua con leve síntomatologia miccional en tratamiento, pero no ha presentado síntomas derivados de su lesión renal."
text5 = "Varón de 68 años, con antecedentes de hemorragia digestiva alta por aspirina y accidente isquémico transitorio a tratamiento crónico con trifusal (300 mg cada 12 horas), que acudió al Servicio de Urgencias del Hospital San Agustín (Avilés, Asturias), en mayo de 2006, por dolor en hemiabdomen izquierdo, intenso, continuo, de instauración súbita y acompañado de cortejo vegetativo. A la exploración presentaba una tensión arterial de 210/120 mm Hg, una frecuencia cardíaca de 80 por minuto, y dolor en fosa ilíaca izquierda, acentuado con la palpación. El hemograma (hemoglobina: 13 g/dL, plaquetas: 249.000), el estudio de coagulación, la bioquímica elemental de sangre, el sistemático de orina, el electrocardiograma y la radiografía simple de tórax eran normales. En la tomografía computarizada de abdomen se objetivó un extenso hematoma, de 12 cm de diámetro máximo, en la celda renal izquierda, sin líquido libre intraperitoneal; la suprarrenal izquierda quedaba englobada y no se podía identificar, y la derecha no presentaba alteraciones. La HTA no se llegó a controlar en Urgencias, a pesar del tratamiento con analgésicos, con antagonistas del calcio y con inhibidores de la enzima convertidora de la angiotensina II, por lo que el paciente, que mantenía cifras tensionales de 240/160 mm Hg, pasó a la Unidad de Cuidados Intensivos, para tratamiento intravenoso con nitroprusiato y labetalol. En las 24 horas siguientes se yuguló la crisis hipertensiva, y se comprobó que la hemoglobina y el hematocrito permanecían estables. Con la sospecha diagnóstica de rotura no traumática de un feocromocitoma pre-existente, se determinaron metanefrinas plasmáticas, que fueron normales, y catecolaminas y metanefrinas urinarias. En la orina de 24 horas del día siguiente al ingreso se obtuvieron los siguientes resultados: adrenalina: 65,1 mcg (valores normales -VN: 1,7-22,5), noradrenalina: 151,1 mcg (VN: 12,1-85,5), metanefrina: 853,5 mcg (VN: 74-297) y normetanefrina: 1396,6 mcg (VN: 105-354). A los 10 días, todavía ingresado el paciente, las cifras urinarias se habían normalizado por completo de modo espontáneo. Respecto al hematoma, en julio de 2006 no se había reabsorbido y persistía una imagen pseudoquística en la zona suprarrenal izquierda. En septiembre de 2006 se practicó una suprarrenalectomía unilateral, y el estudio histológico mostró una masa encapsulada de 6 x 5 cm, con necrosis hemorrágica extensa y algunas células corticales sin atipias."

In [46]:
%%time
extractor = TermExtractor(extraction_methods=["rake"])
extractor(text1)
print(len(extractor.keywords))
extractor(text2)
print(len(extractor.keywords))
extractor(text3)
print(len(extractor.keywords))
extractor(text4)
print(len(extractor.keywords))
extractor(text5)
print(len(extractor.keywords))

55
141
67
52
152
CPU times: user 160 ms, sys: 1.38 ms, total: 161 ms
Wall time: 159 ms


In [42]:
%%time
extractor = TermExtractor(extraction_methods=["yake"])
extractor(text1)
print(len(extractor.keywords))
extractor(text2)
print(len(extractor.keywords))
extractor(text3)
print(len(extractor.keywords))
extractor(text4)
print(len(extractor.keywords))
extractor(text5)
print(len(extractor.keywords))

15
35
19
20
14
CPU times: user 545 ms, sys: 5.49 ms, total: 551 ms
Wall time: 589 ms


In [43]:
%%time
extractor = TermExtractor(extraction_methods=["textrank"])
extractor(text1)
print(len(extractor.keywords))
extractor(text2)
print(len(extractor.keywords))
extractor(text3)
print(len(extractor.keywords))
extractor(text4)
print(len(extractor.keywords))
extractor(text5)
print(len(extractor.keywords))

38
95
52
53
85
CPU times: user 1.04 s, sys: 88.3 ms, total: 1.12 s
Wall time: 1.95 s


In [21]:
%%time
extractor = TermExtractor(extraction_methods=["rake","yake","textrank"], join=True)
extractor(text1)
print(len(extractor.keywords))
extractor(text2)
print(len(extractor.keywords))
extractor(text3)
print(len(extractor.keywords))
extractor(text4)
print(len(extractor.keywords))
extractor(text5)
print(len(extractor.keywords))

68
173
84
76
164
CPU times: user 1.76 s, sys: 97.7 ms, total: 1.86 s
Wall time: 2.46 s


In [22]:
extractor(text1)

In [26]:
a = extractor.keywords[0]
print(a)

<Keyword(text='realizó examen anátomopatológico', method='rake', score='9.0', spans = '[664, 695]')>
